In [1]:
pip install datasets

  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW

In [2]:
cbt_dataset = load_dataset('cbt','CN')

Found cached dataset cbt (C:/Users/34941/.cache/huggingface/datasets/cbt/CN/1.1.0/dc4451a8a4b50cebb78fdb19fa9f964b27fcdcef915467b8b7055a3a8d8cef7b)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
train_set = cbt_dataset['train']
val_set = cbt_dataset['validation']
test_set = cbt_dataset['test']

In [5]:
train_set

Dataset({
    features: ['sentences', 'question', 'answer', 'options'],
    num_rows: 120769
})

In [4]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

C:\Users\34941\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\34941\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


task: write a preprocess function s.t. for an input: context, question, answer, options, output a format that we can put into data loader:
做成input_id, labels, 然后把input_id和labels都padding成一样长就行了应该。

In [ ]:
        context, question, answer, options = self.data[index]
        input_text = f"{context} {question} {' '.join(options)}"
        input_ids = self.tokenizer.encode(input_text, add_special_tokens=True)
        labels = [0] * len(input_ids)
        answer_id = self.tokenizer.encode(answer)[0]
        labels[-len(options) + options.index(answer)] = answer_id
        return torch.tensor(input_ids), torch.tensor(labels)
这个是可以用得上的

In [14]:
context, question, answer, options = train_set[0]['sentences'], train_set[0]['question'], train_set[0]['answer'],train_set[0]['options']

In [25]:
input_text = f"{context} {question} {' '.join(options)}"
input_ids = tokenizer.encode(input_text, add_special_tokens=True)
labels = [0] * len(input_ids)
answer_id = tokenizer.encode(answer)[0]

In [26]:
labels[-len(options) + options.index(answer)] = answer_id

In [28]:
print(torch.tensor(input_ids), torch.tensor(labels))

tensor([17816,  3152,  2048,  2279,  2073,   284,   787,   606,  3772,   837,
          484,  2227,   530,  1517,  1058,   484,   550,   645,  1751,   764,
         3256,   705,  1212, 41548,   276,   262,  5822,   772,   517,   621,
          262, 16599,   837,   508,   373,   845, 14169,   290,  4499,   837,
          290,   508,   550, 16563, 36062,   618,   673,   373,   257,  1200,
          764,  3256,   705,  4864,   837,   673,   837,  1165,   287, 15275,
          286,   477,   262,  3835,   673,  1100,   290,   477,   262,  5986,
          673, 13055,   837,   561,   423,   587,  9675,  1576,   284,   307,
          262,  2802,   286,   257,  1310, 19716,   764,  3256,   705,   464,
         5822,   373, 18116,   284,  5725,   262,  3148,   444,   837,   475,
          262, 16599,   561,   407,  3285,   286,   884,   257,  1517,   764,
         3256,   705,  3347,   750,   407,  1975,   287,  3148,   444,  1058,
          673,   531,   326,   484,   550,  1239, 11196,  2162, 

In [32]:
train_set[0]['answer']

'queen'

In [43]:
class ReadingComprehensionDataset(Dataset):
    def __init__(self, dataset):
        self.data = []
        for example in dataset:
            context = example['sentences']
            question = example['question']
            options = example['options']
            answer = example['answer']
            self.data.append((context, question, answer, options))
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

    def __len__(self):
        return len(self.data)

    def getitem(self, index):
        context, question, answer, options = self.data[index]
        input_text = f"{context} {question} {' '.join(options)}"
        input_ids = self.tokenizer.encode(input_text, add_special_tokens=True)
        labels = [0] * len(input_ids)
        answer_id = self.tokenizer.encode(answer)[0]
        labels[-len(options) + options.index(answer)] = answer_id
        return torch.tensor(input_ids), torch.tensor(labels)

In [44]:
dataset = ReadingComprehensionDataset(train_set)

In [39]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def get_data_loader(dataset, batch_size, shuffle=True):
    def collate_fn(batch):
        input_ids = [torch.tensor(example['input_ids'], dtype=torch.long) for example in batch]
        labels = [torch.tensor(example['label'], dtype=torch.long) for example in batch]
        input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
        labels = pad_sequence(labels, batch_first=True, padding_value=-100) # ignore loss for padding tokens
        return input_ids, labels

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)
    return dataloader


In [40]:
dataset = ReadingComprehensionDataset(train_set)
train_dataloader = get_data_loader(dataset, batch_size=8, shuffle=True)

In [42]:
from transformers import get_linear_schedule_with_warmup

# Define model and optimizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
optimizer = AdamW(model.parameters(), lr=2e-5)

# Define training loop
def train(model, train_dataloader, optimizer, scheduler=None, num_epochs=1):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for input_ids, labels in train_dataloader:
            print(input_ids)
            print(labels)
            input_ids, labels = input_ids.to(device), labels.to(device)
            outputs = model(input_ids, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        print(f"Epoch {epoch+1} Loss: {total_loss/len(train_dataloader)}")

# Load dataset and create dataloader



# Fine-tune model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
num_training_steps = len(train_dataloader) * 3
num_warmup_steps = int(num_training_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)
train(model, train_dataloader, optimizer, scheduler=scheduler, num_epochs=3)

C:\Users\34941\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1050 > 1024). Running this sequence through the model will result in indexing errors


TypeError: tuple indices must be integers or slices, not str